# View CP2K Calculation Results

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.calculation.job.cp2k import Cp2kCalculation
from aiida.orm.code import Code

from aiida.orm import load_node 

import ipywidgets as ipw
from IPython.display import display, clear_output
import nglview

import urlparse
from collections import OrderedDict

In [ ]:
def get_all_calcs():
    # query AiiDA database for structures
    qb = QueryBuilder()
    qb.append(Cp2kCalculation, project=["id", "description"])
    qb.order_by({Cp2kCalculation: {'ctime': 'desc'}})
    #print qb.count()
    qb.all()

    all_calcs = OrderedDict()
    for pk, descr in qb.iterall():
        label = "PK: %d"%pk
        if descr:
            label += "; descr: "+descr
        all_calcs[label] = pk
    return all_calcs

In [ ]:
def reload_calcs(b):
    inp_calc.options = get_all_calcs()
    update_view(None)
    
#inp_calc = ipw.Dropdown(description="Calculation", options=get_all_calcs())
inp_calc = ipw.Select(options=get_all_calcs())
lab_calc = ipw.Label("Calculations:")
btn_reload_cals = ipw.Button(description="Reload")
btn_reload_cals.on_click(reload_calcs)

# parse URL
url = urlparse.urlsplit(jupyter_notebook_url)
querypart = urlparse.parse_qs(url.query)
if 'pk' in querypart.keys():
    inp_calc.value = int(querypart['pk'][0])

display(ipw.HBox([lab_calc, inp_calc, btn_reload_cals]))

In [ ]:
info_box = ipw.HTML()

def update_view(c):
    clear_output()
    if inp_calc.value == None:
        return
    calc = load_node(inp_calc.value)
    output = "pk: %s<br>"%calc.pk
    output += "uuid: %s<br>"%calc.uuid
    output += "state: "+str(calc.get_state())+"<br>\n"
    if calc.get_state() == "WITHSCHEDULER":
        output += "scheduler state: "+str(calc.get_scheduler_state())+"<br>\n"
    
    if calc.get_state() == "FINISHED":
        output += "Energy: %e %s<br>\n"%(calc.res.energy, calc.res.energy_units)
        #pre_block += "Retrieved files:\n   "
        #pre_block += "\n   ".join(sorted(calc.get_retrieved_node().get_folder_list())) + "\n"
        output += "<a target='_blank' href='view_cp2k_orbitals.ipynb?pk=%d'>View orbitals</a><br>"%calc.pk
        rn = calc.get_retrieved_node()
        relpath = rn.folder.abspath.replace("/project/.aiida/repository-default/", "aiida_repository/")
        output += "<a target='_blank' href='/tree/%s/path'>Browser retrieved files</a><br>"%relpath
    output += "<a target='_blank' href='../aiida/aiida_tree_browser.ipynb?pk=%d'>View AiiDA node</a><br>"%calc.pk
    info_box.value = output
    

update_view(None)
inp_calc.observe(update_view, names='value')

display(info_box)